In [8]:
import pandas as pd
def yearly_utilization(directory):
    TankList = [3001, 3002, 3105, 3102, 3103, 3104, 3201, 3202, 3203,3204,3205,3207,3301,3302,3303,3304,3404,3406,3407,3408,3501,3502,3503,3504,3505,3601,3602,3603,3605,3606,3607,4001,4002,4004,4005,4006,4007,4008,4011,4012,4013,4101,4104,4105,4203,4205,4301,4302,4401,4404,4502,4601,4605,4701,4801,4802,4803,4901]
    #TankList = [3202]
    def runtime_cal(Data, levels = False):


            DetailDict = dict() # attribute of equipment ex: running time
            timeList = list() # Time column in Data
            statusList = list() # status column in Data
            status = 0 # status
            recodTime = 0 # recording temp time
            runningTime = 0
            idelTIme = 0
            normal = 1

            for i in range(len(Data)):
                if Data.iat[i, 0] != ' ' or Data.iat[i, 1] != ' ':                
                    timeList.append(Data.iat[i, 0])
                    statusList.append(Data.iat[i,1])
            #print(statusList)

            if type(timeList[0]) == str:
                try:
                    if type(float(timeList[0])) == float:
                        normal = 1
                except ValueError:
                    normal = 0
                    DetailDict["idelTime"] = timeList[0]
                    DetailDict["totalTime"] = timeList[0]
                    DetailDict["runningTime"] = timeList[0]
                    DetailDict["Utilization"] = timeList[0]
            if normal == 1:
                if levels == False:

                    if statusList[0] == "RUNNING" or  statusList[0] == "1":
                        status = 1
                        recodTime = float(timeList[0])
                    else:
                        status = 0
                    for j in range(len(statusList)):
                        if statusList[j] == "RUNNING" or statusList[j] == "1":
                            if status == 0:
                                recodTime = float(timeList[j])
                                status = 1
                            else:
                                pass
                        elif statusList[j] == "STOPPED" or statusList[j] == "0":
                            if status == 0:
                                pass
                            else:
                                runningTime = runningTime + (float(timeList[j]) - recodTime)*24
                                status = 0

                    if statusList[len(statusList)-1] == "RUNNING" or statusList[len(statusList)-1] == "1":
                        runningTime = (float(timeList[len(statusList)-1]) - recodTime)*24 + runningTime
                elif levels == True:

                    try:

                        if float(statusList[0]) > 1.00:
                            status = 1
                            recodTime = float(timeList[0])
                        else:
                            status = 0

                        for j in range(len(statusList)):

                            try:                        
                                if float(statusList[j]) > 1.00:
                                    if status == 0:
                                        recodTime = float(timeList[j])
                                        status = 1
                                        #print(">")
                                    else:
                                        pass
                                elif float(statusList[j]) < 1.00:
                                    if status == 0:
                                        pass
                                    else:
                                        runningTime = runningTime + (float(timeList[j]) - recodTime)*24
                                        status = 0
                            except ValueError:
                                pass
                        if float(statusList[len(statusList)-1]) > 1.00:
                            runningTime =  (float(timeList[len(statusList)-1]) - recodTime)*24 + runningTime
                    except ValueError:
                        pass

                idelTime = (float(timeList[len(timeList)-1])  - float(timeList[0]))*24 - runningTime
                totalTime = (float(timeList[len(timeList)-1]) - float(timeList[0]))*24


                DetailDict["idelTime"] = idelTime
                DetailDict["totalTime"] = totalTime
                DetailDict["runningTime"] = runningTime
                DetailDict["Utilization"] = runningTime/totalTime

                #print("Total Time: " + str(totalTime/24) + " days")
                #print("Overall Idle Time:" + str(idelTime) + " hr")
                #print("Overall Running Time: " + str(runningTime) + " hr")
                #print("Overall Utilization: " + str(runningTime/totalTime))
                #print()
            return DetailDict

    Tankdcit = dict() # overall Tank
    for i in range(len(TankList)):
        Filename  = directory + "Tank " + str(TankList[i]) + ".csv"
        data = pd.read_csv(Filename)

        agi = data[["AgiTime", "Agitator Status"]]
        level = data[["LevelTime", "Tank Level"]]
        jack = data[["JackPTime", "Tank Jacket Pump"]]
        samp = data[["SamPTime", "Tank Sample Pump"]]
        con = data[["ConPTime", "Condenser Circulation Pump"]]

        agi = agi.dropna()
        level = level.dropna()
        jack = jack.dropna()
        samp = samp.dropna()
        con = con.dropna()


        TankEquipDict = dict() # tank equipment Dict ex: agitator, level


        TankEquipDict["Agitator"] = runtime_cal(agi)
        TankEquipDict["TankLevel"] = runtime_cal(level, levels = True)
        TankEquipDict["JacketPump"] = runtime_cal(jack)
        TankEquipDict["SamplePump"] = runtime_cal(samp)
        TankEquipDict["CondenserPump"] = runtime_cal(con)


        Tankdcit[TankList[i]] = TankEquipDict


    Colname = ["Tank No", "Agitator Running Time", "Agitator Idle Time", "Agitator Utilization Rate", "Tank Level Running Time", "Tank Level Idle Time", "Tank Level Utilization Rate",
                  "JacketPump Running Time", "JacketPump idle Time", "JacketPump Utilization Rate", "SamplePump Running Time", "SamplePump Idle Time", "SamplePump Utilization Rate",
                  "CondenserPump Running Time", "CondenserPump Idle Time", "CondenserPump Utilization Rate"]
    report_file = pd.DataFrame(columns = Colname )
    for i in Tankdcit:
        tempList = list()
        tempList.append(i)
        tempList.append(Tankdcit[i]['Agitator']['runningTime'])
        tempList.append(Tankdcit[i]['Agitator']['idelTime'])
        tempList.append(Tankdcit[i]['Agitator']['Utilization'])
        tempList.append(Tankdcit[i]['TankLevel']['runningTime'])
        tempList.append(Tankdcit[i]['TankLevel']['idelTime'])
        tempList.append(Tankdcit[i]['TankLevel']['Utilization'])
        tempList.append(Tankdcit[i]['JacketPump']['runningTime'])
        tempList.append(Tankdcit[i]['JacketPump']['idelTime'])
        tempList.append(Tankdcit[i]['JacketPump']['Utilization'])
        tempList.append(Tankdcit[i]['SamplePump']['runningTime'])
        tempList.append(Tankdcit[i]['SamplePump']['idelTime'])
        tempList.append(Tankdcit[i]['SamplePump']['Utilization'])
        tempList.append(Tankdcit[i]['CondenserPump']['runningTime'])
        tempList.append(Tankdcit[i]['CondenserPump']['idelTime'])
        tempList.append(Tankdcit[i]['CondenserPump']['Utilization'])
        c = pd.Series(tempList, index = Colname)
        report_file = report_file.append(c, ignore_index=True)
    
    return report_file

def output_report(directory, file):
    import pandas as pd
    directory = directory + 'xxx.csv'
    file.to_csv(directory)